## Baseball Prediction: 3b - Augment DataFrame with Odds Data
In the previous notebook we got historical odds data from oddsshark.com and saved them as a set of csv files (with a particular naming convention).

In this notebook we will load that data and augment our primary (game-level) data frame so that it includes this odds data - specifically, the implied probabilities and the over/under, for each game.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lxml
import html5lib
%matplotlib inline
from urllib.request import urlopen
import time
import structureboost as stb
import ml_insights as mli
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

## Load in our game level data

In [2]:
df=pd.read_csv('df_bp1.csv')

/Users/brianlucena/.pyenv/versions/3.9.13/envs/ds3913/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (13,14,15,85,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Plan of attack
- Create a dictionary structure to enable us to easily go to a specific game for a particular team and season.
- Iterate through the rows of our main dataframe, look up the home and visiting team for that game to get their odds. (Note: need two different lookups per game)

## Create a dictionary for easy lookup

In [3]:
# Use the same mapping as before
# (more elegant to save to file...)

oddsshark_num_to_team_dict = {}
oddsshark_num_to_team_dict[26995]='PHI'
oddsshark_num_to_team_dict[26996]='SDN'
oddsshark_num_to_team_dict[26997]='SFN'
oddsshark_num_to_team_dict[26998]='ANA'
oddsshark_num_to_team_dict[26999]='DET'
oddsshark_num_to_team_dict[27000]='CIN'
oddsshark_num_to_team_dict[27001]='NYA'
oddsshark_num_to_team_dict[27002]='TEX'
oddsshark_num_to_team_dict[27003]='TBA'
oddsshark_num_to_team_dict[27004]='COL'
oddsshark_num_to_team_dict[27005]='MIN'
oddsshark_num_to_team_dict[27006]='KCA'
oddsshark_num_to_team_dict[27007]='ARI'
oddsshark_num_to_team_dict[27008]='BAL'
oddsshark_num_to_team_dict[27009]='ATL'
oddsshark_num_to_team_dict[27010]='TOR'
oddsshark_num_to_team_dict[27011]='SEA'
oddsshark_num_to_team_dict[27012]='MIL'
oddsshark_num_to_team_dict[27013]='PIT'
oddsshark_num_to_team_dict[27014]='NYN'
oddsshark_num_to_team_dict[27015]='LAN'
oddsshark_num_to_team_dict[27016]='OAK'
oddsshark_num_to_team_dict[27017]='WAS'
oddsshark_num_to_team_dict[27018]='CHA'
oddsshark_num_to_team_dict[27019]='SLN'
oddsshark_num_to_team_dict[27020]='CHN'
oddsshark_num_to_team_dict[27021]='BOS'
oddsshark_num_to_team_dict[27022]='MIA'
oddsshark_num_to_team_dict[27023]='HOU'
oddsshark_num_to_team_dict[27024]='CLE'

In [4]:
# Use the saved files to get the odds information
# We create a dict based on team and season for easy lookup
df_odds_dict={}
for i in range(26995, 27025):
    team_name = oddsshark_num_to_team_dict[i]
    df_odds_dict[team_name] = {}
    print(team_name)
    for season in range(2019,2023):
        fname = 'oddsshark_'+team_name+'_'+str(season)+'.csv'
        df_temp = pd.read_csv(fname)
        df_temp['date_dblhead'] = (df_temp.date_numeric.astype(str) + df_temp.dblheader_num.astype(str)).astype(int)
        df_temp.set_index('date_dblhead', inplace=True)
        df_odds_dict[team_name][season] = df_temp

PHI
SDN
SFN
ANA
DET
CIN
NYA
TEX
TBA
COL
MIN
KCA
ARI
BAL
ATL
TOR
SEA
MIL
PIT
NYN
LAN
OAK
WAS
CHA
SLN
CHN
BOS
MIA
HOU
CLE


### Augment our main DataFrame

In [5]:
# Again, we iterate through the main dataframe
# get the team, season, game and then get
# the relevant info from the odds dictionary

implied_prob_h = np.zeros(df.shape[0])
implied_prob_v = np.zeros(df.shape[0])
over_under = np.zeros(df.shape[0])
ou_result = np.full(df.shape[0],'', dtype=object)
for ind, row in df.iterrows():
    if (ind%1000)==0:
        print(ind)
    if row.season<2019:
        continue
    else:
        season = row['season']
        home_team = row['team_h']
        visit_team = row['team_v']
        home_game_no = row['game_no_h']
        visit_game_no = row['game_no_v']
        date_dblh = row['date_dblhead']
        try:
            implied_prob_h[ind] = df_odds_dict[home_team][season].loc[date_dblh,'prob_implied']
            over_under[ind] = df_odds_dict[home_team][season].loc[date_dblh,'Total']
            ou_result[ind] = df_odds_dict[home_team][season].loc[date_dblh,'OU']
        except KeyError:
            print(f'Game not found wrt home_team:{home_team} vs {visit_team} date_dbl {date_dblh}')
        try:
            implied_prob_v[ind] = df_odds_dict[visit_team][season].loc[date_dblh,'prob_implied']
        except KeyError:
            print(f'Game not found wrt visit_team:{visit_team} vs {home_team} date_dbl {date_dblh}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
Game not found wrt home_team:CHA vs SEA date_dbl 202106260
Game not found wrt visit_team:SEA vs CHA date_dbl 202106260
Game not found wrt home_team:ATL vs SDN date_dbl 202107211
Game not found wrt visit_team:SDN vs ATL date_dbl 202107211
Game not found wrt home_team:ATL vs SDN date_dbl 202107212
Game not found wrt visit_team:SDN vs ATL date_dbl 202107212
93000
94000
95000
96000


In [6]:
df['implied_prob_h'] = implied_prob_h
df['implied_prob_v'] = implied_prob_v
df['implied_prob_h_mid'] = (implied_prob_h + (1-implied_prob_v))/2
df['over_under_line']=over_under
df['over_under_result']=ou_result

### Drop 3 games with no odds

In [7]:
df[(df.season>=2019) & (df.implied_prob_h==0)]

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result
92555,20210626,0,Sat,SEA,AL,78,CHA,AL,76,3,2,54,D,"20210627,CHI12,0,0,15",NaN,NaN,CHI12,30017.0,197,000011001,000011000,34,7,0,0,3,3,0,0,0,3,0,9,0,0,1,0,7,7,2,2,0,0,27,9,0,0,3,0,29,6,0,0,0,2,0,1,1,3,0,11,1,0,3,0,5,3,3,3,0,0,27,7,1,0,1,0,cuzzp901,Phil Cuzzi,hallt901,Tom Hallion,rippm901,Mark Ripperger,blasc901,Cory Blaser,NaN,(none),NaN,(none),servs002,Scott Servais,larut101,Tony LaRussa,sewap001,Paul Sewald,hendl001,Liam Hendriks,gravk001,Kendall Graveman,tramt001,Taylor Trammell,gilbl002,Logan Gilbert,lynnl001,Lance Lynn,crawj002,J.P. Crawford,6,hanim001,Mitch Haniger,9,seagk001,Kyle Seager,5,frant002,Ty France,3,bauej001,Jake Bauers,7,longs001,Shed Long,4,torrl001,Luis Torrens,2,fralj001,Jake Fraley,10,tramt001,Taylor Trammell,8,andet001,Tim Anderson,6,goodb001,Brian Goodwin,8,moncy001,Yoan Moncada,5,abrej003,Jose Abreu,3,grany001,Yasmani Grandal,2,lambj001,Jake Lamb,7,mercy001,Yermin Mercedes,10,garcl004,Leury Garcia,4,gonzl005,Luis Gonzalez,9,"umpchange,3,ump1b,clemp901",Y,2021,-1,0,5,202106260,0.261093,0.220639,0.327324,0.292443,0.433627,0.372102,0.760951,0.664545,56.0,104.0,27.0,34.0,102.0,81.0,0.239378,0.243539,0.314019,0.301194,0.383420,0.410537,0.697438,0.711731,11.0,15.0,7.0,5.0,18.0,15.0,0.0,0.0,0.5,0.0,
92838,20210721,1,Wed,SDN,NL,98,ATL,NL,94,3,2,42,D,NaN,NaN,NaN,ATL03,28621.0,156,0001200,0000020,26,7,2,0,1,3,0,1,0,4,0,6,0,1,0,0,7,3,1,1,0,0,21,10,1,1,0,0,25,5,1,0,0,1,1,1,0,0,0,3,0,0,0,0,4,3,3,3,2,0,21,8,0,0,0,0,timmt901,Tim Timmons,riggj901,Jeremy Riggs,mahrn901,Nick Mahrley,marqa901,Alfonso Marquez,NaN,(none),NaN,(none),tingj801,Jayce Tingler,snitb801,Brian Snitker,paddc001,Chris Paddack,mullk001,Kyle Muller,melam001,Mark Melancon,kim-h002,Ha-Seong Kim,paddc001,Chris Paddack,mullk001,Kyle Muller,phamt001,Tommy Pham,7,tatif002,Fernando Tatis,6,cronj001,Jake Cronenworth,3,machm001,Manny Machado,5,myerw001,Wil Myers,9,profj001,Jurickson Profar,8,kim-h002,Ha-Seong Kim,4,rivaw00

In [8]:
indices_to_drop = df[(df.season>=2019) & (df.implied_prob_h==0)].index
indices_to_drop

Int64Index([92555, 92838, 92839], dtype='int64')

In [9]:
df.shape

(96276, 199)

In [10]:
df.drop(indices_to_drop, inplace=True)
df.shape

(96273, 199)

In [11]:
df.reset_index(inplace=True, drop=True)
df.shape

(96273, 199)

In [12]:
df.to_csv('df_bp3.csv', index=False)